Käesolev Jupyter Notebook on TÖR päringu masinloetava vastuse **töötlemiseks**.

In [15]:
import os
import sqlite3
import sys
from datetime import datetime, timedelta, date
import pandas as pd
from xml.etree import ElementTree
import plotly.figure_factory as ff



print("        # #                ")
print("#####   ###   ####          #              ##    #              ##                   #                  ")
print("  #    #   #  #   #         #               #    #               #                   #                  ")
print("  #    #   #  #   #         #   #   ###     #    #   #  #   #    #     ###    ###   ####    ###   # ##  ")
print("  #    #   #  ####          #  #       #    #    #  #   #   #    #        #      #   #     #   #  ##  # ")
print("  #    #   #  # #           ###     ####    #    ###    #   #    #     ####   ####   #     #   #  #     ")
print("  #    #   #  #  #          #  #   #   #    #    #  #   #  ##    #    #   #  #   #   #  #  #   #  #     ")
print("  #     ###   #   #         #   #   ####   ###   #   #   ## #   ###    ####   ####    ##    ###   #     ")
print("\nAutor: maiko punkt motsar [ät] politsei punkt ee")
#print("Versioon: v.0.0.1 08.08.2021 | NB: rakendus kogub kasutusstatistikat päringute arvu kohta. Sellega on tehtud "+str(kasutuskordi)+" päringut.")

print("\nKleebi misp2 TÖR päringu masinloetav vastus ja vajuta 2x <ENTER>:\n")

sisend = []

while True:
    rida = input()
    if rida:
        sisend.append(rida)
    else:
        break

multilineSisend = '\n'.join(sisend)


sisendPuu = ElementTree.fromstring(multilineSisend)

# define namespace mappings to use as shorthand below
namespaces = {
    'soap': 'http://schemas.xmlsoap.org/soap/envelope/',
    'a': 'http://x-road.eu/xsd/representation.xsd',
}

for päring in sisendPuu.findall('.//request'):
    päringuSisend = {}
    päringuSisend['isikukoodid'] = päring.find('isikukoodid').text
    # print(päringuSisend['isikukoodid'])

## väljastame päringu vastusest välja päringu sisendi andmed
print("Päringu sisend: " + sisendPuu.find('.//request').find(
    'isikukoodid').text + "\nTöötamise algus: " + datetime.strptime(
    sisendPuu.find('.//request').find('toot_algus').text, "%Y-%m-%d").strftime(
    "%d.%m.%Y") + "\nTöötamise lõpp: " + datetime.strptime(sisendPuu.find('.//request').find('toot_lopp').text,
                                                           "%Y-%m-%d").strftime("%d.%m.%Y") + "\n\n")

## loome hulga, kuhu kogume kokku kõikide töötamiste päevad
kõikidelRegistreeringutelTöötamisePäevad = set()

töötatudPäevadeHulkIlmaPeatamisteta = set()
for leitud in sisendPuu.findall('.//kanne'):

    ## loome sõnastiku ühe tsüklikorra kannete hoiustamiseks
    kirjeKanded = {}

    ##loome ühe registriingu päevade loendamise hulga
    registreeringulTöötatudPäevad = set()

    kirjeKanded['toot_id'] = leitud.find('toot_id').text
    kirjeKanded['isikukood'] = leitud.find('isikukood').text
    kirjeKanded['synniaeg'] = leitud.find('synniaeg').text
    kirjeKanded['perenimi'] = leitud.find('perenimi').text
    kirjeKanded['eesnimi'] = leitud.find('eesnimi').text
    kirjeKanded['kodakondsus_kood'] = leitud.find('kodakondsus_kood').text
    kirjeKanded['kodakondsus_nimi'] = leitud.find('kodakondsus_nimi').text
    kirjeKanded['ta_kood'] = leitud.find('ta_kood').text
    kirjeKanded['ta_nimi'] = leitud.find('ta_nimi').text
    kirjeKanded['tootamise_liik_kood'] = leitud.find('tootamise_liik_kood').text
    kirjeKanded['tootamise_liik_nimi'] = leitud.find('tootamise_liik_nimi').text
    if (leitud.find('leping')):
        kirjeKanded['leping'] = leitud.find('leping').text
    if (leitud.find('tootamise_lopetamise_alus_kood')):
        kirjeKanded['tootamise_lopetamise_alus_kood'] = leitud.find('tootamise_lopetamise_alus_kood').text
    if (leitud.find('tootamise_lopetamise_alus_nimi')):
        kirjeKanded['tootamise_lopetamise_alus_nimi'] = leitud.find('tootamise_lopetamise_alus_nimi').text
    if (leitud.find('riigi_kood')):
        kirjeKanded['riigi_kood'] = leitud.find('riigi_kood').text
    kirjeKanded['kande_olek_kood'] = leitud.find('kande_olek_kood').text
    kirjeKanded['kande_olek_nimi'] = leitud.find('kande_olek_nimi').text
    if (leitud.find('ta_markus')):
        kirjeKanded['ta_markus'] = leitud.find('ta_markus').text
    if (leitud.find('mta_markus')):
        kirjeKanded['mta_markus'] = leitud.find('mta_markus').text
    kirjeKanded['sisestamise_aeg'] = leitud.find('sisestamise_aeg').text
    kirjeKanded['muutmise_aeg'] = leitud.find('muutmise_aeg').text
    if (leitud.find('reg_tel_nr')):
        kirjeKanded['reg_tel_nr'] = leitud.find('reg_tel_nr').text
    kirjeKanded['parsimisMärkus'] = ''

    kirjeKanded['tootamise_algus'] = datetime.strptime(leitud.find('tootamise_algus').text, "%Y-%m-%d")

    if (leitud.find('tootamise_lopp') != None):
        kirjeKanded['tootamise_lopp'] = datetime.strptime(leitud.find('tootamise_lopp').text, "%Y-%m-%d")
    else:
        kirjeKanded['tootamise_lopp'] = datetime.strptime("2021-08-07", "%Y-%m-%d")
        kirjeKanded['parsimisMärkus'] = kirjeKanded[
                                            'parsimisMärkus'] + "| HOIATUS: lõpukuupäev puudu → TÄNAne kuupäeva lõpukuupäevana ! "
        ##print("hoiatus: kandel polnud lõpukuupäeva ja asendasin selle tänase kuupäevaga")

    ## loome ühest registreeringu algus ja lõpukuupäevast Pandase date range
    registreeringuDateRange = pd.date_range(start=kirjeKanded['tootamise_algus'], end=kirjeKanded['tootamise_lopp'])

    ## itereerime läbi kõik ühe registreeringu Date Range kuupäevad ja lisame need hulka registreeringulTöötatudPäevad kujul pp.kk.aaaa
    for i in registreeringuDateRange:
        registreeringulTöötatudPäevad.add(i.strftime("%d.%m.%Y"))

    ### siia tuleb mahaarvamiste arvutus

    ## kontrollime, kas kanne sisaldab peatamisi. Kui jah, siis kogume need kokku eraldi listi
    peatamisteList = []
    peatamised = leitud.findall('.//peatamiste_jada/peatamine')
    if peatamised is not None and len(peatamised) != 0:
        peatatudPäevadeHulk = set()
        for töötamisPaus in peatamised:
            peatusKanne = {}
            peatusKanne['peat_id'] = töötamisPaus.find('peat_id').text
            peatusKanne['peat_algus'] = datetime.strptime(töötamisPaus.find('peat_algus').text, "%Y-%m-%d")
            peatusKanne['peat_alus_kood'] = töötamisPaus.find('peat_alus_kood').text
            peatusKanne['peat_lopp'] = datetime.strptime(töötamisPaus.find('peat_lopp').text, "%Y-%m-%d")
            peatusKanne['peat_alus_nimi'] = töötamisPaus.find('peat_alus_nimi').text
            peatusKanne['kestvus'] = len(pd.date_range(start=peatusKanne['peat_algus'], end=peatusKanne['peat_lopp']))

            ## lisame peatamise päevad kujul pp.kk.aaaa kõiki ühe registreeringu peatamisi hõlmavasse hulka
            for i in pd.date_range(start=peatusKanne['peat_algus'], end=peatusKanne['peat_lopp'], normalize=True):
                peatatudPäevadeHulk.add(i.strftime("%d.%m.%Y"))
            ## print(len(peatatudPäevadeHulk),peatatudPäevadeHulk)
            peatamisteList.append(peatusKanne)
            del (peatusKanne)

        ## võtame registreeringulTöötatudPäevad hulgast maha peatamiste hulga ja kustutame selle igaks juhuks seejärel ära
        ##print("Kontroll, registreeringul töötatud päevade hulk ENNE peatamiste maha arvamist: ",len(registreeringulTöötatudPäevad))
        registreeringulTöötatudPäevad = registreeringulTöötatudPäevad - peatatudPäevadeHulk
        del (peatatudPäevadeHulk)

        ##print("Kontroll, registreeringul töötatud päevade hulk peale peatamiste maha arvamist: ",len(registreeringulTöötatudPäevad), registreeringulTöötatudPäevad)

        kirjeKanded['peatamiskanded'] = peatamisteList

    ## ühe kande tegelikult töötatud päevade hulga lisamine kõikide töötamiste päevade hulgale
    kõikidelRegistreeringutelTöötamisePäevad = kõikidelRegistreeringutelTöötamisePäevad | registreeringulTöötatudPäevad
    ### kas on vajalik registreeringuTöötatudPäevad kustutamine või tühjendamine???

    ##print("Töötatud päevade vahekokkuvõte: ", len(kõikidelRegistreeringutelTöötamisePäevad))
    os.system('color')
    tekstiVärviAlgus = "\x1b[33m"
    if (kirjeKanded['kande_olek_nimi']=="Kehtiv"):
        tekstiVärviAlgus="\x1b[32m"

    tekstiVärviReset = "\x1b[0m"

    print(tekstiVärviAlgus+kirjeKanded['toot_id'] + '\t' + kirjeKanded['tootamise_algus'].strftime("%d.%m.%Y") + "\t" + kirjeKanded[
        'tootamise_lopp'].strftime("%d.%m.%Y") + "\t" + str(len(registreeringuDateRange))+ "\t" + str(len(registreeringulTöötatudPäevad)) + "\t" + kirjeKanded[
              'kande_olek_nimi'] + "\t" + kirjeKanded['ta_nimi'] + "\t" + kirjeKanded['ta_kood'] + "\t\x1b[31m" + kirjeKanded[
              'parsimisMärkus']+'\x1b[0m')
    if ("peatamiskanded" in kirjeKanded):

        for i in kirjeKanded['peatamiskanded']:
            print('\x1b[93m→' + i['peat_id'] + "\t" + i['peat_algus'].strftime("%d.%m.%Y") + "\t" + i['peat_lopp'].strftime(
                "%d.%m.%Y") + "\t-" + str(i['kestvus'])+"\t0" + "\t" + i['peat_alus_nimi']+'\x1b[0m')

## kõikidelRegistreeringutelTöötamisePäevad hulgast kõikide KORDUMATUTE töötamispäevade summa väljastamine
print("\nSisendandmete põhjal on isik PÄRINGUPERIOODIL (peale töötamiste peatamiste maha arvamist) töötanud: ",
      len(kõikidelRegistreeringutelTöötamisePäevad), " mittekattuvat (kalendri)päeva.")


perioodiKuupäev = datetime.strptime(input(
    "\nKas asetame eelnevad töötamisajad 365/455-päevasese perioodi? Sisesta perioodi algus- või lõpukuupäev kujul PP.KK.AAAA "),
                                    "%d.%m.%Y")
if (perioodiKuupäev != None):

    ##loome 455 päevase perioodi ettepoole
    ette455 = pd.date_range(end=perioodiKuupäev, periods=455, normalize=True)
    hulkEtte455 = set()
    for i in ette455:
        hulkEtte455.add(i.strftime("%d.%m.%Y"))
    # print(hulkEtte455)
    print("\n\n455-päevane tagasiulatuv periood\t" + ette455[0].strftime("%d.%m.%Y") + "\t" + ette455[-1].strftime(
        "%d.%m.%Y") + "\t, millest töötati " + str(len(
        kõikidelRegistreeringutelTöötamisePäevad & hulkEtte455)) + " kalendripäeva. Jääk (365 - " + str(
        len((kõikidelRegistreeringutelTöötamisePäevad & hulkEtte455))) + ") on " + str(
        365 - len((kõikidelRegistreeringutelTöötamisePäevad & hulkEtte455))) + " päeva.\n")
    # print("\n455-päevane tagasiulatuv periood:\t" + ette455[0].strftime("%d.%m.%Y") + "\t" + ette455[-1].strftime(
    #     "%d.%m.%Y"))
    # print("Kõikidest töötamise päevadest jääb 455-päevase perioodi ", ette455[0].strftime("%d.%m.%Y"), "\t",
    #       ette455[-1].strftime("%d.%m.%Y"), " sisse ", len((kõikidelRegistreeringutelTöötamisePäevad & hulkEtte455)),
    #       " kalendripäeva. Kasutamata jääk ( 365 - ", len((kõikidelRegistreeringutelTöötamisePäevad & hulkEtte455)),
    #       ") on ", 365 - len((kõikidelRegistreeringutelTöötamisePäevad & hulkEtte455)), " päeva.\n")

    # print(hulkEtte455 - töötatudPäevadeHulkIlmaPeatamisteta)

    ##loome 455 päevase perioodi edasi poole
    edasi455 = pd.date_range(start=perioodiKuupäev, periods=455)
    hulkEdasi455 = set()
    for i in edasi455:
        hulkEdasi455.add(i.strftime("%d.%m.%Y"))
    print("455-päevane edasiulatuv periood\t" + edasi455[0].strftime("%d.%m.%Y") + "\t" + edasi455[-1].strftime(
        "%d.%m.%Y") + "\t, millest töötati " + str(len(
        kõikidelRegistreeringutelTöötamisePäevad & hulkEdasi455)) + " kalendripäeva. Jääk (365 - " + str(
        len((kõikidelRegistreeringutelTöötamisePäevad & hulkEdasi455))) + ") on " + str(
        365 - len((kõikidelRegistreeringutelTöötamisePäevad & hulkEdasi455))) + " päeva.\n")
    #print("Kõikidest töötamise päevadest jääb 455-päevase perioodi ", edasi455[0].strftime("%d.%m.%Y"), "\t",
    #edasi455[-1].strftime("%d.%m.%Y"), " sisse ", len((kõikidelRegistreeringutelTöötamisePäevad & hulkEdasi455)),
    #" kalendripäeva. Kasutamata jääk ( 365 - ", len((kõikidelRegistreeringutelTöötamisePäevad & hulkEdasi455)), ") on ",
    #365 - len((kõikidelRegistreeringutelTöötamisePäevad & hulkEdasi455)), " päeva.\n")

    ##loome 365 päevase perioodi ettepoole
    ette365 = pd.date_range(end=perioodiKuupäev, periods=365)
    hulkEtte365 = set()
    for i in ette365:
        hulkEtte365.add(i.strftime("%d.%m.%Y"))
    print("365-päevane tagasiulatuv periood\t" + ette365[0].strftime("%d.%m.%Y") + "\t" + ette365[-1].strftime(
        "%d.%m.%Y") + "\t, millest töötati " + str(len(
        kõikidelRegistreeringutelTöötamisePäevad & hulkEtte365)) + " kalendripäeva. Jääk (270 - " + str(
        len((kõikidelRegistreeringutelTöötamisePäevad & hulkEtte365))) + ") on " + str(
        270 - len((kõikidelRegistreeringutelTöötamisePäevad & hulkEtte365))) + " päeva.\n")
    # print("365-päevane tagasiulatuv periood:\t" + ette365[0].strftime("%d.%m.%Y") + "\t" + ette365[-1].strftime(
    #     "%d.%m.%Y"))
    # print("Kõikidest töötamise päevadest jääb 455-päevase perioodi ", ette365[0].strftime("%d.%m.%Y"), "\t",
    # ette365[-1].strftime("%d.%m.%Y"), " sisse ", len((kõikidelRegistreeringutelTöötamisePäevad & hulkEtte365)),
    # " kalendripäeva. Kasutamata jääk ( 270 - ", len((kõikidelRegistreeringutelTöötamisePäevad & hulkEtte365)), ") on ",
    # 270 - len((kõikidelRegistreeringutelTöötamisePäevad & hulkEtte365)), " päeva.\n")

    ##loome 365 päevase perioodi edasi poole
    edasi365 = pd.date_range(start=perioodiKuupäev, periods=365)
    hulkEdasi365 = set()
    for i in edasi365:
        hulkEdasi365.add(i.strftime("%d.%m.%Y"))
    print("365-päevane edasiulatuv periood\t" + edasi365[0].strftime("%d.%m.%Y") + "\t" + edasi365[-1].strftime(
        "%d.%m.%Y") + "\t, millest töötati " + str(len(
        kõikidelRegistreeringutelTöötamisePäevad & hulkEdasi365)) + " kalendripäeva. Jääk (270 - " + str(
        len((kõikidelRegistreeringutelTöötamisePäevad & hulkEdasi365))) + ") on " + str(
        270 - len((kõikidelRegistreeringutelTöötamisePäevad & hulkEdasi365))) + " päeva.\n")
    # print("365-päevane edasiulatuv periood:\t" + edasi365[0].strftime("%d.%m.%Y") + "\t" + edasi365[-1].strftime(
    #     "%d.%m.%Y"))
    # print("Kõikidest töötamise päevadest jääb 455-päevase perioodi ", edasi365[0].strftime("%d.%m.%Y"), "\t",
    # edasi365[-1].strftime("%d.%m.%Y"), " sisse ", len((kõikidelRegistreeringutelTöötamisePäevad & hulkEdasi365)),
    # " kalendripäeva. Kasutamata jääk ( 270 - ", len((kõikidelRegistreeringutelTöötamisePäevad & hulkEdasi365)), ") on ",
    # 270 - len((kõikidelRegistreeringutelTöötamisePäevad & hulkEdasi365)), " päeva.\n")



    if (input("\nKas väljastame andmed CSV faili? (J/E) ").upper() == "J"):
        failiNimi = "TÖR_" + kirjeKanded['perenimi'].title() + "_" + kirjeKanded[
            'eesnimi'].title() + "_" + datetime.now().strftime("%Y%m%d%H%M%S") + ".csv"
        f = open(failiNimi.replace(' ','_'), "a", encoding="UTF-8")
        f.write("test" + "\n")
        f.write("test")
        f.close()
        print("Faili '" + failiNimi.replace(' ','_') + "' kirjutamine lõppes edukalt. Leiad selle skriptiga samast kaustast")
        os.system(failiNimi.replace(' ','_'))
        del (failiNimi)

    ##print(leitud)

    ##print(sisendPuu)

    #input()

        # #                
#####   ###   ####          #              ##    #              ##                   #                  
  #    #   #  #   #         #               #    #               #                   #                  
  #    #   #  #   #         #   #   ###     #    #   #  #   #    #     ###    ###   ####    ###   # ##  
  #    #   #  ####          #  #       #    #    #  #   #   #    #        #      #   #     #   #  ##  # 
  #    #   #  # #           ###     ####    #    ###    #   #    #     ####   ####   #     #   #  #     
  #    #   #  #  #          #  #   #   #    #    #  #   #  ##    #    #   #  #   #   #  #  #   #  #     
  #     ###   #   #         #   #   ####   ###   #   #   ## #   ###    ####   ####    ##    ###   #     

Autor: maiko punkt motsar [ät] politsei punkt ee

Kleebi misp2 TÖR päringu masinloetav vastus ja vajuta 2x <ENTER>:

<SOAP-ENV:Envelope xmlns:SOAP-ENV="http://schemas.xmlsoap.org/soap/envelope/" xmlns:SOAP-ENC="http://schemas.xmlsoap.or

In [ ]:
print("Kas soovid perioodid kuvada diagrammil? J/E")


In [20]:
data = [{'Task': '14.05.2019 - 14.06.2019\t32', 'Start': '2019-05-14 00:00:00', 'Finish': '2019-06-14 00:00:00', 'Resource': '14.05.2019 - 14.06.2019', 'Description': 'valikuline täiendav kirjeldus'}, {'Task': '12.07.2019 - 03.11.2019\t115', 'Start': '2019-07-12 00:00:00', 'Finish': '2019-11-03 00:00:00', 'Resource': '12.07.2019 - 03.11.2019', 'Description': 'valikuline täiendav kirjeldus'}, {'Task': '13.02.2020 - 15.08.2020\t185', 'Start': '2020-02-13 00:00:00', 'Finish': '2020-08-15 00:00:00', 'Resource': '13.02.2020 - 15.08.2020', 'Description': 'valikuline täiendav kirjeldus'}, {'Task': '455 päeva edasi', 'Start': '2021-09-14', 'Finish': '2022-12-12', 'Resource': 'Ajaraam edasi'}, {'Task': '455 päeva tagasi', 'Start': '2020-06-17', 'Finish': '2021-09-14', 'Resource': 'Ajaraam tagasi'}, {'Task': '365 päeva edasi', 'Start': '2021-09-14', 'Finish': '2022-09-13', 'Resource': 'Ajaraam edasi'}, {'Task': '365 päeva tagasi', 'Start': '2020-09-15', 'Finish': '2021-09-14', 'Resource': 'Ajaraam tagasi'}, {'Task': '270 päeva edasi', 'Start': '2021-09-14', 'Finish': '2022-06-10', 'Resource': 'Ajaraam edasi'}, {'Task': '270 päeva tagasi', 'Start': '2020-12-19', 'Finish': '2021-09-14', 'Resource': 'Ajaraam tagasi'}]

fig = ff.create_gantt(data,title="Ajavahemike Gantti diagramm tänase "+' seisuga',index_col='Resource', show_colorbar=True,group_tasks=True,showgrid_y=True,show_hover_fill=True)
fig.update_layout(hovermode="x unified", height=400)
fig.show()